# Buffering

In [ ]:
import geopandas
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [12, 8]

In [ ]:
triangle = ((0,0), (2,2), (4,0))
from  shapely.geometry import Polygon, Point

In [ ]:
triangle = Polygon(triangle)

In [ ]:
triangle

In [ ]:
buffer = triangle.buffer(1)

In [ ]:
buffer

In [ ]:
point = Point(4.5, 0)

In [ ]:
point

In [ ]:
point.intersects(triangle)

In [ ]:
point.intersects(buffer)

In [ ]:
point_1 = Point((10,10))
point_2 = Point((20, 10))

In [ ]:
poly_1 = point_1.buffer(6)
poly_2 = point_2.buffer(6)

In [ ]:
poly_1

In [ ]:
type(poly_1)

In [ ]:
type(point_1)

In [ ]:
poly_2.contains(point_1)

In [ ]:
poly_1.contains(point_1)

## Clinics and Tracts 

In [ ]:
clinics_df = geopandas.read_file('data/behavioralHealth.shp')

In [ ]:
clinics_df.plot()

In [ ]:
clinics_df.crs

In [ ]:
tracts_df = geopandas.read_file('data/california_tracts.shp')

In [ ]:
tracts_df.plot()

In [ ]:
tracts_df.crs

In [ ]:
tracts_df.to_crs(clinics_df.crs).plot()

In [ ]:
projected_tracts_df = tracts_df.to_crs(clinics_df.crs)

In [ ]:
projected_tracts_df.crs

In [ ]:
projected_tracts_df.head()

FIPS: https://www.weather.gov/hnx/cafips

In [ ]:
riverside_tracts = projected_tracts_df[projected_tracts_df.GEOID10.str.startswith('06065')]

In [ ]:
riverside_tracts.plot()

In [ ]:
centroids = riverside_tracts.geometry.centroid # tract centroids

In [ ]:
centroids.plot()

In [ ]:
base = centroids.plot()
clinics_df.plot(ax=base)

In [ ]:
clinic_buffer_3000 = clinics_df.buffer(3000)

In [ ]:
base = clinic_buffer_3000.plot()
clinics_df.plot(ax=base, color='red')

In [ ]:
clinic_buffer_3000.geometry

In [ ]:
base = clinic_buffer_3000.plot()
clinics_df.plot(ax=base, color='red')

## What tracts intersect with one or more clinic buffers?

In [ ]:
centroids = geopandas.GeoDataFrame(geometry=centroids)
clinic_buffer_3000 = geopandas.GeoDataFrame(geometry=clinic_buffer_3000)
centroids_intersecting = geopandas.sjoin(centroids, clinic_buffer_3000, how='inner', op='intersects')

In [ ]:
centroids_intersecting.head()

In [ ]:
centroids_intersecting.shape

In [ ]:
tracts_intersecting = geopandas.sjoin(riverside_tracts, clinic_buffer_3000, how='inner', op='intersects')

In [ ]:
tracts_intersecting.head()

In [ ]:
tracts_intersecting.shape

There are 132 cases of a tract intersecting a clinic buffer

## Create a new attribute for the tracts reporting the number of clinics within 3000 feet of the tract polygon

In [ ]:
tracts_intersecting.groupby(by='GEOID10').count()

In [ ]:
riverside_tracts.merge(tracts_intersecting.groupby(by='GEOID10').count()['index_right'], on="GEOID10")

In [ ]:
riverside_tracts.merge(tracts_intersecting.groupby(by='GEOID10').count()['index_right'], on="GEOID10", how='left')

In [ ]:
riverside_tracts.shape

So the left merge makes sure all the original tracts are included in the merge, even if they do not intersect with a buffer. In those cases a `NaN` is inserted.

We can replace the `NaN` values with `0`s.

In [ ]:
riverside_tracts.merge(tracts_intersecting.groupby(by='GEOID10').count()['index_right'], on="GEOID10", how='left').fillna(0)

In [ ]:
riverside_tracts.merge(tracts_intersecting.groupby(by='GEOID10').count()['index_right'], on="GEOID10", how='left').fillna(0).rename(columns={'index_right':'clinics'})

In [ ]:
riverside_tracts_clinics = riverside_tracts.merge(tracts_intersecting.groupby(by='GEOID10').count()['index_right'], on="GEOID10", how='left').fillna(0).rename(columns={'index_right':'clinics'})

In [ ]:
riverside_tracts_clinics.plot(column='clinics', scheme='Quantiles')

In [ ]:
riverside_tracts_clinics.plot(column='clinics', scheme='FisherJenks', k=5, legend=True)

## Order matters

Note that it could be that a single clinic buffer intersects with more than a single tract.

In [ ]:
clinics_intersecting = geopandas.sjoin(clinic_buffer_3000, riverside_tracts, how='left', op='intersects')

In [ ]:
clinics_intersecting.shape

In [ ]:
clinics_df.shape

There are only 28 clinics, each with a buffer. But there are 132 intersections involving a buffer and a tract

In [ ]:
clinics_intersecting.head()

here the index is the index of the clinic buffer, and we see the first index, 0, repeats 4 times.

In [ ]:
clinics_intersecting['clinic'] = clinics_intersecting.index
clinics_intersecting.reset_index(level=0, inplace=True)

In [ ]:
clinics_intersecting.head()

In [ ]:
clinics_intersecting.groupby(by='index').count()

In [ ]:
clinics_intersecting = clinics_intersecting.groupby(by='index').count()

In [ ]:
clinics_intersecting.head()

In [ ]:
clinics_intersecting['tracts'] = clinics_intersecting.index_right

In [ ]:
clinics_intersecting.head()

In [ ]:
clinics_df.merge(clinics_intersecting[['tracts']], left_index=True, right_index=True)

In [ ]:
clinics_df = clinics_df.merge(clinics_intersecting[['tracts']], left_index=True, right_index=True)

In [ ]:
clinics_df.plot(column='tracts', scheme='Quantiles', k=3, legend=True)